<a href="https://colab.research.google.com/github/Parv17k/nyc_collision_data_analysis/blob/main/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

In [3]:
!wget -O longlat.csv -q https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&timezone=America/New_York&lang=en&use_labels_for_header=true&csv_separator=%3B

In [4]:
ls

longlat.csv  sample_data/  spark-3.1.1-bin-hadoop2.7.tgz


In [5]:
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

In [6]:
!pip install -q findspark

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [8]:
import findspark
findspark.init()

In [9]:
import findspark
findspark.init()

In [10]:
findspark.find()

'/content/spark-3.1.1-bin-hadoop2.7'

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Collision")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [12]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -y ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-03-31 20:47:09--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.70.180.11, 54.226.184.31, 54.197.133.26, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.70.180.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14746350 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.06M  18.1MB/s    in 0.8s    

2021-03-31 20:47:10 (18.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14746350/14746350]

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  test 

In [13]:
!wget -q https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv ## Here we are downloading dataset for Collisions in New York City

In [19]:
mv rows.csv col.csv ## Linux cmd to rename dataset that we got in the above command.

In [18]:
ls

longlat.csv                   rows.csv      spark-3.1.1-bin-hadoop2.7/
ngrok-stable-linux-amd64.zip  sample_data/  spark-3.1.1-bin-hadoop2.7.tgz


In [15]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
import pyspark
conf = pyspark.SparkConf()
conf.set("spark.driver.memory","8G");
import numpy as np
sc = SparkContext.getOrCreate(conf=conf)
from pyspark.sql.functions import round, col


In [16]:
spark =SparkSession(sc)

In [20]:

# Reading data from local repo. to pyspark.

col=spark.read.option("header",True).csv("./col.csv",inferSchema=True) # data set 1.
longlat=spark.read.option("delimiter",";").csv("./longlat.csv",inferSchema=True,header=True) #location. [Additional Dataset ]

In [21]:
col.count() - col.distinct().count() # checking for Duplicate rows in our dataset.

0

In [22]:
len(col.columns) # how many columns do I have.

29

In [23]:
col.printSchema() #Schema of my dataset. Issues with our columns types like NUMBER OF PERSONS INJURED: string and It should need to be Integer.

root
 |-- CRASH DATE: string (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = true)
 |-- CROSS STREET NAME: string (nullable = true)
 |-- OFF STREET NAME: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: string (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: string (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: string (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 2: strin

In [24]:
from pyspark.sql.types import IntegerType

col=col.withColumn("NUMBER OF PERSONS INJURED", col["NUMBER OF PERSONS INJURED"].cast(IntegerType()))
col=col.withColumn("NUMBER OF CYCLIST KILLED", col["NUMBER OF CYCLIST KILLED"].cast(IntegerType()))
col=col.withColumn("NUMBER OF MOTORIST INJURED", col["NUMBER OF MOTORIST INJURED"].cast(IntegerType()))

# Correcting the column types to Integer which were previously of String types.

In [25]:
col.printSchema() # checking if my schema is correct now ? Yes It is

root
 |-- CRASH DATE: string (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = true)
 |-- CROSS STREET NAME: string (nullable = true)
 |-- OFF STREET NAME: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: integer (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: integer (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: integer (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 2: st

In [26]:
col.select("CRASH DATE").show(10)

+----------+
|CRASH DATE|
+----------+
|05/06/2013|
|05/08/2013|
|05/06/2013|
|05/07/2013|
|05/07/2013|
|05/09/2013|
|05/08/2013|
|05/09/2013|
|05/05/2013|
|05/08/2013|
+----------+
only showing top 10 rows



In [27]:
#dropping columns with collision id = null

In [28]:
col.filter(col.COLLISION_ID.isNull()).drop()

DataFrame[CRASH DATE: string, CRASH TIME: string, BOROUGH: string, ZIP CODE: string, LATITUDE: double, LONGITUDE: double, LOCATION: string, ON STREET NAME: string, CROSS STREET NAME: string, OFF STREET NAME: string, NUMBER OF PERSONS INJURED: int, NUMBER OF PERSONS KILLED: int, NUMBER OF PEDESTRIANS INJURED: int, NUMBER OF PEDESTRIANS KILLED: int, NUMBER OF CYCLIST INJURED: int, NUMBER OF CYCLIST KILLED: int, NUMBER OF MOTORIST INJURED: int, NUMBER OF MOTORIST KILLED: int, CONTRIBUTING FACTOR VEHICLE 1: string, CONTRIBUTING FACTOR VEHICLE 2: string, CONTRIBUTING FACTOR VEHICLE 3: string, CONTRIBUTING FACTOR VEHICLE 4: string, CONTRIBUTING FACTOR VEHICLE 5: string, COLLISION_ID: int, VEHICLE TYPE CODE 1: string, VEHICLE TYPE CODE 2: string, VEHICLE TYPE CODE 3: string, VEHICLE TYPE CODE 4: string, VEHICLE TYPE CODE 5: string]

In [29]:
col.count()

1766160

In [30]:
missingLocation=col.filter(col["BOROUGH"].isNull()).filter(col["ZIP CODE"].isNull())

In [31]:
cannotRetriveLocation = missingLocation.filter(col["LONGITUDE"].isNull())

In [32]:
col.count() ## removed the records for which we cannot retrive location at any cost.

1766160

In [33]:
#Infer the schema, and register the DataFrame as a table.
col.createOrReplaceTempView("col");
cannotRetriveLocation.createOrReplaceTempView("toDelete")

In [34]:
col=spark.sql("select * FROM col WHERE col.COLLISION_ID not in (select COLLISION_ID from toDelete where COLLISION_ID is not null)");


In [35]:
col.count()

1588740

In [36]:
col.filter(col["ZIP CODE"].isNull()).filter(col["BOROUGH"].isNotNull()).count()

211

In [37]:
BOROUGH=list(map(lambda x : x["BOROUGH"],col.select("BOROUGH").distinct().collect()))

In [38]:
BOROUGH

[None, 'QUEENS', 'BROOKLYN', 'BRONX', 'MANHATTAN', 'STATEN ISLAND']

In [39]:
longlat.printSchema() # another data set

root
 |-- zip: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- timezone: integer (nullable = true)
 |-- dst: integer (nullable = true)
 |-- geopoint: string (nullable = true)



In [40]:
col.createOrReplaceTempView("col");
longlat.createOrReplaceTempView("longlat");

In [41]:
missingZipCode=col.filter(col['ZIP CODE'].isNull());

In [ ]:
missingZipCode.createOrReplaceTempView("missingZipCode");


In [42]:
col.select("NUMBER OF CYCLIST INJURED").distinct().show(); ## lets check.

+-------------------------+
|NUMBER OF CYCLIST INJURED|
+-------------------------+
|                        1|
|                        3|
|                        4|
|                        2|
|                        0|
+-------------------------+



NUMBER OF CYCLIST INJURED Is alredy cleaned.

In [43]:
col.select("NUMBER OF PERSONS INJURED").distinct().show();

+-------------------------+
|NUMBER OF PERSONS INJURED|
+-------------------------+
|                       31|
|                       27|
|                       12|
|                       22|
|                     null|
|                        1|
|                       13|
|                        6|
|                       16|
|                        3|
|                       20|
|                        5|
|                       19|
|                       15|
|                       43|
|                        9|
|                       17|
|                        4|
|                        8|
|                        7|
+-------------------------+
only showing top 20 rows



NUMBER OF PERSONS INJURED has Null value hence making it to Zero.

In [44]:
col=col.fillna(value=0,subset=["NUMBER OF PERSONS INJURED"]) # clean null values to zero.

In [45]:
#NUMBER OF PERSONS KILLED
print(col.select("NUMBER OF PERSONS KILLED").distinct().show());

+------------------------+
|NUMBER OF PERSONS KILLED|
+------------------------+
|                    null|
|                       1|
|                       3|
|                       5|
|                       4|
|                       8|
|                       2|
|                       0|
+------------------------+

None


NUMBER OF PERSONS KILLED has NULL values : replacign them with 0.


In [46]:
col=col.fillna(value=0,subset=["NUMBER OF PERSONS KILLED"])

In [47]:
print(col.select("NUMBER OF PEDESTRIANS INJURED").distinct().show());

+-----------------------------+
|NUMBER OF PEDESTRIANS INJURED|
+-----------------------------+
|                           27|
|                            1|
|                           13|
|                            6|
|                            3|
|                            5|
|                           15|
|                            9|
|                            4|
|                            7|
|                            2|
|                            0|
+-----------------------------+

None


NUMBER OF PEDESTRIANS INJURED has no null value

In [48]:
#NUMBER OF PEDESTRIANS KILLED
print(col.select("NUMBER OF PEDESTRIANS KILLED").distinct().show());

+----------------------------+
|NUMBER OF PEDESTRIANS KILLED|
+----------------------------+
|                           1|
|                           6|
|                           2|
|                           0|
+----------------------------+

None




```
NUMBER OF PEDESTRIANS KILLED has no null value
```



In [49]:
print(col.select("NUMBER OF CYCLIST INJURED").distinct().show());

+-------------------------+
|NUMBER OF CYCLIST INJURED|
+-------------------------+
|                        1|
|                        3|
|                        4|
|                        2|
|                        0|
+-------------------------+

None


NUMBER OF CYCLIST INJURED has no null value

In [50]:
print(col.select("NUMBER OF CYCLIST KILLED").distinct().show());

+------------------------+
|NUMBER OF CYCLIST KILLED|
+------------------------+
|                       1|
|                       2|
|                       0|
+------------------------+

None


NUMBER OF CYCLIST KILLED has no null value

In [51]:
#NUMBER OF MOTORIST INJURED
print(col.select("NUMBER OF MOTORIST INJURED").distinct().show(100));

+--------------------------+
|NUMBER OF MOTORIST INJURED|
+--------------------------+
|                        31|
|                        12|
|                        22|
|                         1|
|                        13|
|                         6|
|                        16|
|                         3|
|                        20|
|                         5|
|                        19|
|                        15|
|                        43|
|                         9|
|                        17|
|                         4|
|                         8|
|                         7|
|                        10|
|                        24|
|                        21|
|                        11|
|                        14|
|                         2|
|                        30|
|                         0|
|                        18|
+--------------------------+

None


NUMBER OF MOTORIST INJURED has no null value

In [52]:
#NUMBER OF MOTORIST INJURED
print(col.select("NUMBER OF MOTORIST KILLED").distinct().show(100));

+-------------------------+
|NUMBER OF MOTORIST KILLED|
+-------------------------+
|                        1|
|                        3|
|                        5|
|                        4|
|                        2|
|                        0|
+-------------------------+

None


NUMBER OF MOTORIST INJURED has no null value

In [53]:
#CONTRIBUTING FACTOR VEHICLE
print(col.select("CONTRIBUTING FACTOR VEHICLE 1").distinct().show(70));

# has numerical values 1 and 80
# has null value.


+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 1|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|         Accelerator Defec...|
|         Driverless/Runawa...|
|         Windshield Inadeq...|
|         Using On Board Na...|
|                 Unsafe Speed|
|         Shoulders Defecti...|
|               Tinted Windows|
|            Oversized Vehicle|
|         Passing or Lane U...|
|         Lane Marking Impr...|
|         Other Lighting De...|
|                         null|
|         Aggressive Drivin...|
|              Other Vehicular|
|          Driver Inexperience|
|          Tow Hitch Defective|
|                      Texting|
|                       Illnes|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Prescription Medi...|
|         Cell Phone (hand-...|
|         View Obstructed/L...|
|           Lost Consciousness|
|         Reaction to Other...|
|       

In [54]:
#CONTRIBUTING FACTOR VEHICLE
print(col.select("CONTRIBUTING FACTOR VEHICLE 2").distinct().show(100));

# has numerical values 1 and 80.
# has null value.


+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 2|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|         Driverless/Runawa...|
|         Accelerator Defec...|
|         Windshield Inadeq...|
|         Using On Board Na...|
|                 Unsafe Speed|
|         Shoulders Defecti...|
|               Tinted Windows|
|            Oversized Vehicle|
|         Passing or Lane U...|
|         Lane Marking Impr...|
|         Other Lighting De...|
|                         null|
|         Aggressive Drivin...|
|              Other Vehicular|
|          Driver Inexperience|
|          Tow Hitch Defective|
|                      Texting|
|                       Illnes|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Prescription Medi...|
|         Cell Phone (hand-...|
|         View Obstructed/L...|
|           Lost Consciousness|
|         Reaction to Other...|
|       

In [55]:
#CONTRIBUTING FACTOR VEHICLE
print(col.select("CONTRIBUTING FACTOR VEHICLE 3").distinct().show(100));

# has numerical values 1 and 80.
# has null value.

+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 3|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|         Driverless/Runawa...|
|         Accelerator Defec...|
|                 Unsafe Speed|
|               Tinted Windows|
|            Oversized Vehicle|
|         Passing or Lane U...|
|         Lane Marking Impr...|
|                         null|
|         Aggressive Drivin...|
|              Other Vehicular|
|          Driver Inexperience|
|                       Illnes|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Prescription Medi...|
|         View Obstructed/L...|
|           Lost Consciousness|
|         Reaction to Other...|
|         Reaction to Uninv...|
|                  Fell Asleep|
|         Tire Failure/Inad...|
|         Outside Car Distr...|
|              Drugs (Illegal)|
|              Fatigued/Drowsy|
|         Driver Inattentio...|
|       

In [56]:
#CONTRIBUTING FACTOR VEHICLE
print(col.select("CONTRIBUTING FACTOR VEHICLE 4").distinct().show(100));
# has null value.

+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 4|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|         Accelerator Defec...|
|         Driverless/Runawa...|
|         Windshield Inadeq...|
|                 Unsafe Speed|
|         Passing or Lane U...|
|                         null|
|         Aggressive Drivin...|
|              Other Vehicular|
|          Driver Inexperience|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Prescription Medi...|
|         View Obstructed/L...|
|           Lost Consciousness|
|         Reaction to Uninv...|
|                  Fell Asleep|
|         Outside Car Distr...|
|              Drugs (Illegal)|
|              Fatigued/Drowsy|
|         Driver Inattentio...|
|         Failure to Keep R...|
|           Obstruction/Debris|
|         Unsafe Lane Changing|
|          Alcohol Involvement|
|                      Illness|
|       

In [57]:
print(col.select("CONTRIBUTING FACTOR VEHICLE 5").distinct().show(30));
# has null values

+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 5|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|                 Unsafe Speed|
|         Passing or Lane U...|
|                         null|
|         Aggressive Drivin...|
|              Other Vehicular|
|          Driver Inexperience|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Reaction to Uninv...|
|                  Fell Asleep|
|         Tire Failure/Inad...|
|         Outside Car Distr...|
|              Fatigued/Drowsy|
|         Driver Inattentio...|
|         Failure to Keep R...|
|           Obstruction/Debris|
|          Alcohol Involvement|
|                      Illness|
|         Traffic Control D...|
|             Brakes Defective|
|             Backing Unsafely|
|         Failure to Yield ...|
|                        Glare|
|             Steering Failure|
|          Passing Too Closely|
|       

In [58]:
col.count()

1588740

In [59]:
## Setting all null values to Unknown 
col=col.fillna(value="Unknown",subset=["CONTRIBUTING FACTOR VEHICLE 1","CONTRIBUTING FACTOR VEHICLE 2","CONTRIBUTING FACTOR VEHICLE 3","CONTRIBUTING FACTOR VEHICLE 4","CONTRIBUTING FACTOR VEHICLE 5"])

In [60]:
## Checking what values in VEHICLE TYPE CODE need to be cleaned
print(col.select("VEHICLE TYPE CODE 1","VEHICLE TYPE CODE 2","VEHICLE TYPE CODE 3","VEHICLE TYPE CODE 4","VEHICLE TYPE CODE 5").distinct().show(30));


+--------------------+--------------------+--------------------+--------------------+--------------------+
| VEHICLE TYPE CODE 1| VEHICLE TYPE CODE 2| VEHICLE TYPE CODE 3| VEHICLE TYPE CODE 4| VEHICLE TYPE CODE 5|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|             UNKNOWN|             UNKNOWN|                null|                null|                null|
|               Sedan|               MOPED|                null|                null|                null|
|           Box Truck|Station Wagon/Spo...|          Motorcycle|               Sedan|Station Wagon/Spo...|
|           Lift Boom|           Box Truck|                null|                null|                null|
|          SKID LOADE|Station Wagon/Spo...|                null|                null|                null|
|             TRAILER|       Pick-up Truck|                null|                null|                null|
|         Chassis Cab|Station Wagon/S

In [61]:
## Setting null values for vehicle type to unknown
col=col.fillna(value="Unknown",subset=["VEHICLE TYPE CODE 1","VEHICLE TYPE CODE 2","VEHICLE TYPE CODE 3","VEHICLE TYPE CODE 4","VEHICLE TYPE CODE 5"])

In [62]:
## Checking if the values were correctly changed
print(col.select("VEHICLE TYPE CODE 1","VEHICLE TYPE CODE 2","VEHICLE TYPE CODE 3","VEHICLE TYPE CODE 4","VEHICLE TYPE CODE 5").distinct().show(30));

+--------------------+--------------------+--------------------+--------------------+--------------------+
| VEHICLE TYPE CODE 1| VEHICLE TYPE CODE 2| VEHICLE TYPE CODE 3| VEHICLE TYPE CODE 4| VEHICLE TYPE CODE 5|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|                DELV|               Sedan|             Unknown|             Unknown|             Unknown|
|           Box Truck|Station Wagon/Spo...|          Motorcycle|               Sedan|Station Wagon/Spo...|
|               Sedan|Multi-Wheeled Veh...|             Unknown|             Unknown|             Unknown|
|                 van|             Unknown|             Unknown|             Unknown|             Unknown|
|           Carry All|           Carry All|           Carry All|             Unknown|             Unknown|
|                 Bus|               Sedan|Tractor Truck Diesel|             Unknown|             Unknown|
|                 van|Station Wagon/S

In [63]:
### Set all empty values in CROSS STREET NAME to No Value
col=col.fillna(value="NOT PROVIDED",subset=["CROSS STREET NAME"])

In [64]:
### Set all empty values in OFF STREET NAME to No Value
col=col.fillna(value="NOT PROVIDED",subset=["OFF STREET NAME"])

In [65]:
### Set all empty values in ON STREET NAME to No Value
col=col.fillna(value="NOT PROVIDED",subset=["ON STREET NAME"])

In [158]:
## Checking if results have been successfully changed
print(col.select("CROSS STREET NAME","OFF STREET NAME","ON STREET NAME").distinct().show(30));

+--------------------+--------------------+--------------------+
|   CROSS STREET NAME|     OFF STREET NAME|      ON STREET NAME|
+--------------------+--------------------+--------------------+
|NEILSON STREET   ...|        NOT PROVIDED|CORNAGA AVENUE   ...|
|FOSTER AVENUE    ...|        NOT PROVIDED|EAST 87 STREET   ...|
|WEST 66 STREET   ...|        NOT PROVIDED|BROADWAY         ...|
|BELMONT AVENUE   ...|        NOT PROVIDED|SNEDIKER AVENUE  ...|
|JAMAICA AVENUE   ...|        NOT PROVIDED|FRANCIS LEWIS BOU...|
|PARSONS BOULEVARD...|        NOT PROVIDED|77 ROAD          ...|
|        NOT PROVIDED|        NOT PROVIDED|GREENE AVENUE    ...|
|        NOT PROVIDED|674       AUTUMN ...|        NOT PROVIDED|
|        NOT PROVIDED|1219      FLATBUS...|        NOT PROVIDED|
|            9 AVENUE|        NOT PROVIDED|50 STREET        ...|
|        LENOX AVENUE|        NOT PROVIDED|WEST 132 STREET  ...|
|        NOT PROVIDED|427       WEST 26...|        NOT PROVIDED|
|        NOT PROVIDED|108

In [67]:
#Check CRASH DATE format
col.filter(col["CRASH DATE"].isNull()).show()

+----------+----------+-------+--------+--------+---------+--------+--------------+-----------------+---------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME|BOROUGH|ZIP CODE|LATITUDE|LONGITUDE|LOCATION|ON STREET NAME|CROSS STREET NAME|OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBUTING FACTOR VEHICLE 1|CONTRIBUTING FACTOR VEHICLE 2|CONTRIBUTING F

In [ ]:
#Change CRASH DATE type from String to Date
from pyspark.sql.types import DateType
from pyspark.sql.functions import to_date

#col.withColumn("CRASH DATE", col['CRASH DATE'].cast(DateType())).show()

 
col= col.withColumn('CRASH DATE',to_date(col["CRASH DATE"], 'MM/dd/yyyy'))
col.printSchema()

root
 |-- CRASH DATE: date (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = false)
 |-- CROSS STREET NAME: string (nullable = false)
 |-- OFF STREET NAME: string (nullable = false)
 |-- NUMBER OF PERSONS INJURED: integer (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: integer (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: integer (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = false)
 |-- CONTRIBUTING FACTOR VEHICLE 2: 

In [ ]:
# Check if format is correct
col.select("CRASH DATE").show(10)

+----------+
|CRASH DATE|
+----------+
|2018-10-15|
|2018-11-01|
|2018-10-23|
|2018-11-03|
|2018-10-15|
|2018-11-03|
|2018-11-04|
|2018-10-19|
|2018-10-12|
|2018-10-27|
+----------+
only showing top 10 rows



In [68]:
## Converting CRASH TIME type: String to timestamp 
from pyspark.sql.types import DateType
from pyspark.sql.functions import to_timestamp

col= col.withColumn('CRASH TIME',to_timestamp(col["CRASH TIME"], 'H:mm'))
col.filter(col["CRASH TIME"].isNull()).show()

+----------+----------+-------+--------+--------+---------+--------+--------------+-----------------+---------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME|BOROUGH|ZIP CODE|LATITUDE|LONGITUDE|LOCATION|ON STREET NAME|CROSS STREET NAME|OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBUTING FACTOR VEHICLE 1|CONTRIBUTING FACTOR VEHICLE 2|CONTRIBUTING F

In [ ]:
## Checking if format is correct
col.select("CRASH TIME").show(10)

+-------------------+
|         CRASH TIME|
+-------------------+
|1970-01-01 08:00:00|
|1970-01-01 16:30:00|
|1970-01-01 20:12:00|
|1970-01-01 14:35:00|
|1970-01-01 17:30:00|
|1970-01-01 20:40:00|
|1970-01-01 13:50:00|
|1970-01-01 12:10:00|
|1970-01-01 18:20:00|
|1970-01-01 14:45:00|
+-------------------+
only showing top 10 rows



In [ ]:
### Removing YYYY-MM-DD from CRASH TIME
from pyspark.sql.types import DateType
from pyspark.sql.functions import date_format
from pyspark.sql import functions as f

col = col.withColumn("CRASH TIME", f.date_format("CRASH TIME", 'HH:mm:ss'))


In [ ]:
## Checking Results of Format
col.select("CRASH TIME").show(10)

+----------+
|CRASH TIME|
+----------+
|  08:00:00|
|  16:30:00|
|  20:12:00|
|  14:35:00|
|  17:30:00|
|  20:40:00|
|  13:50:00|
|  12:10:00|
|  18:20:00|
|  14:45:00|
+----------+
only showing top 10 rows



In [ ]:
col.show()

+----------+----------+-------------+--------+---------+----------+--------------------+--------------------+-------------------+--------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+--------------------+--------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME|      BOROUGH|ZIP CODE| LATITUDE| LONGITUDE|            LOCATION|      ON STREET NAME|  CROSS STREET NAME|     OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRI

In [70]:
col.write.csv('mycsv.csv')

# ***TODO***
Treat Null Values for all column to  Unknown. 1,2,3,4,5


In [71]:
# zip,borough,location,long,lat, [contributing1,2,3,4], killed and injured [3-4],COLLISION_ID.

In [74]:
ls


col.csv      ngrok-stable-linux-amd64.zip  spark-3.1.1-bin-hadoop2.7.tgz
longlat.csv  sample_data/
mycsv.csv/   spark-3.1.1-bin-hadoop2.7/


In [75]:
!wget -q https://data.cityofnewyork.us/api/views/f55k-p6yu/rows.csv   #Loaded person data for vehicle collisions

In [78]:
mv rows.csv person.csv

In [80]:
ls

col.csv      ngrok-stable-linux-amd64.zip  spark-3.1.1-bin-hadoop2.7/
longlat.csv  person.csv                    spark-3.1.1-bin-hadoop2.7.tgz
mycsv.csv/   sample_data/


In [81]:
person = spark.read.option("header",True).csv("./person.csv",inferSchema=True) # Read person data.

In [82]:
person.createOrReplaceTempView("person");

In [83]:
person.count()

4236940

In [84]:
len(person.columns)

21

In [85]:
person.printSchema()

root
 |-- UNIQUE_ID: integer (nullable = true)
 |-- COLLISION_ID: integer (nullable = true)
 |-- CRASH_DATE: string (nullable = true)
 |-- CRASH_TIME: string (nullable = true)
 |-- PERSON_ID: string (nullable = true)
 |-- PERSON_TYPE: string (nullable = true)
 |-- PERSON_INJURY: string (nullable = true)
 |-- VEHICLE_ID: integer (nullable = true)
 |-- PERSON_AGE: integer (nullable = true)
 |-- EJECTION: string (nullable = true)
 |-- EMOTIONAL_STATUS: string (nullable = true)
 |-- BODILY_INJURY: string (nullable = true)
 |-- POSITION_IN_VEHICLE: string (nullable = true)
 |-- SAFETY_EQUIPMENT: string (nullable = true)
 |-- PED_LOCATION: string (nullable = true)
 |-- PED_ACTION: string (nullable = true)
 |-- COMPLAINT: string (nullable = true)
 |-- PED_ROLE: string (nullable = true)
 |-- CONTRIBUTING_FACTOR_1: string (nullable = true)
 |-- CONTRIBUTING_FACTOR_2: string (nullable = true)
 |-- PERSON_SEX: string (nullable = true)



In [86]:
person.count() - person.distinct().count() 

0

In [87]:
person.filter(person.COLLISION_ID.isNull()).drop()

DataFrame[UNIQUE_ID: int, COLLISION_ID: int, CRASH_DATE: string, CRASH_TIME: string, PERSON_ID: string, PERSON_TYPE: string, PERSON_INJURY: string, VEHICLE_ID: int, PERSON_AGE: int, EJECTION: string, EMOTIONAL_STATUS: string, BODILY_INJURY: string, POSITION_IN_VEHICLE: string, SAFETY_EQUIPMENT: string, PED_LOCATION: string, PED_ACTION: string, COMPLAINT: string, PED_ROLE: string, CONTRIBUTING_FACTOR_1: string, CONTRIBUTING_FACTOR_2: string, PERSON_SEX: string]

In [88]:
person.filter(person.UNIQUE_ID.isNull()).drop() 

DataFrame[UNIQUE_ID: int, COLLISION_ID: int, CRASH_DATE: string, CRASH_TIME: string, PERSON_ID: string, PERSON_TYPE: string, PERSON_INJURY: string, VEHICLE_ID: int, PERSON_AGE: int, EJECTION: string, EMOTIONAL_STATUS: string, BODILY_INJURY: string, POSITION_IN_VEHICLE: string, SAFETY_EQUIPMENT: string, PED_LOCATION: string, PED_ACTION: string, COMPLAINT: string, PED_ROLE: string, CONTRIBUTING_FACTOR_1: string, CONTRIBUTING_FACTOR_2: string, PERSON_SEX: string]

In [89]:
person.filter(person.PERSON_ID.isNull()).count()

19

In [90]:
person=person.fillna(value='Unknown',subset=["PERSON_ID"])

In [91]:
print(person.select("PERSON_TYPE").distinct().show(100))

+-----------+
|PERSON_TYPE|
+-----------+
|   Occupant|
| Pedestrian|
|  Bicyclist|
+-----------+

None


In [92]:
print(person.select("PERSON_INJURY").distinct().show(100)) #Already Clean

+-------------+
|PERSON_INJURY|
+-------------+
|       Killed|
|  Unspecified|
|      Injured|
+-------------+

None


In [93]:
person.select("VEHICLE_ID").distinct().count()

1939149

In [94]:
person.filter(person.VEHICLE_ID.isNull()).count()

165791

In [95]:

person=person.fillna(value='-999',subset=["VEHICLE_ID"])

In [96]:
person.filter(person.PERSON_AGE.isNull()).count()

363520

In [97]:

mean = spark.sql("Select sum(PERSON_AGE)/count(PERSON_AGE) from person where PERSON_AGE > 0 and PERSON_AGE < 100");

In [98]:
mean.show()

+-------------------------------------------------------------------------------------+
|(CAST(sum(CAST(PERSON_AGE AS BIGINT)) AS DOUBLE) / CAST(count(PERSON_AGE) AS DOUBLE))|
+-------------------------------------------------------------------------------------+
|                                                                    40.24500953358535|
+-------------------------------------------------------------------------------------+



In [99]:
mean.createOrReplaceTempView("mean")

In [100]:
mean = spark.sql("SELECT cast((select * from mean) as int)")

In [101]:
from pyspark.sql.functions import when

In [102]:
person = person.withColumn('PERSON_AGE', when(person["PERSON_AGE"] > 100, 40).otherwise(person["PERSON_AGE"]))

In [103]:
person = person.withColumn('PERSON_AGE', when(person["PERSON_AGE"] < 0, 40).otherwise(person["PERSON_AGE"]))

In [104]:
person=person.fillna(value=40,subset=["PERSON_AGE"])

In [107]:
person.select("PERSON_AGE").distinct().show()

+----------+
|PERSON_AGE|
+----------+
|        31|
|        85|
|        65|
|        53|
|        78|
|        34|
|        81|
|        28|
|        76|
|        27|
|        26|
|        44|
|        12|
|        91|
|        22|
|        93|
|        47|
|         1|
|        52|
|        13|
+----------+
only showing top 20 rows



In [106]:
person.select("EJECTION").distinct().show()

+-----------------+
|         EJECTION|
+-----------------+
|Partially Ejected|
|   Does Not Apply|
|             null|
|          Unknown|
|      Not Ejected|
|          Ejected|
|          Trapped|
+-----------------+



In [108]:
person=person.fillna(value='Unspecified',subset=["EJECTION"])  #Cleaned EJECTED

In [109]:
person.select("EMOTIONAL_STATUS").distinct().show()

+----------------+
|EMOTIONAL_STATUS|
+----------------+
|      Incoherent|
|  Does Not Apply|
|   Semiconscious|
|            null|
|         Unknown|
|     Unconscious|
|           Shock|
|  Apparent Death|
|       Conscious|
+----------------+



In [110]:
person=person.fillna(value='Unspecified',subset=["EMOTIONAL_STATUS"])  #Cleaned EMOTIONAL_STATUS

In [111]:
person.select("BODILY_INJURY").distinct().show()

+--------------------+
|       BODILY_INJURY|
+--------------------+
|      Does Not Apply|
|                null|
|                Head|
|             Unknown|
|                Neck|
|Elbow-Lower-Arm-Hand|
|         Entire Body|
|                Face|
|                Back|
|       Hip-Upper Leg|
|    Abdomen - Pelvis|
|                 Eye|
|Shoulder - Upper Arm|
|               Chest|
| Knee-Lower Leg Foot|
+--------------------+



In [112]:
person=person.fillna(value='Unspecified',subset=["BODILY_INJURY"])

In [113]:
person.select("PERSON_SEX").distinct().show()

+----------+
|PERSON_SEX|
+----------+
|         F|
|      null|
|         M|
|         U|
+----------+



In [114]:

person=person.fillna(value='Unspecified',subset=["PERSON_SEX"])  #Cleaned PERSON_SEX

In [115]:
person.select("POSITION_IN_VEHICLE").distinct().show()

+--------------------+
| POSITION_IN_VEHICLE|
+--------------------+
|Riding/Hanging on...|
|Front passenger, ...|
|Any person in the...|
|If one person is ...|
|      Does Not Apply|
|              Driver|
|Right rear passen...|
|                null|
|Middle front seat...|
|             Unknown|
|Left rear passeng...|
|Middle rear seat,...|
+--------------------+



In [116]:
person=person.fillna(value='Unspecified',subset=["POSITION_IN_VEHICLE"])

In [117]:
person.select("SAFETY_EQUIPMENT").distinct().show()

+--------------------+
|    SAFETY_EQUIPMENT|
+--------------------+
|                None|
|Stoppers Only (In...|
|Pads Only (In-Lin...|
|                null|
|    Air Bag Deployed|
|Air Bag Deployed/...|
|Air Bag Deployed/...|
|             Unknown|
|               Other|
|                   -|
|             Harness|
|Helmet/Other (In-...|
|Child Restraint Only|
|    Lap Belt/Harness|
|Helmet (Motorcycl...|
|  Lap Belt & Harness|
|Helmet Only (In-L...|
|Air Bag Deployed/...|
|            Lap Belt|
+--------------------+



In [118]:
person=person.fillna(value='Unspecified',subset=["SAFETY_EQUIPMENT"])

In [119]:
person.select("PED_LOCATION").distinct().show()

+--------------------+
|        PED_LOCATION|
+--------------------+
|      Does Not Apply|
|                null|
|             Unknown|
|Pedestrian/Bicycl...|
|Pedestrian/Bicycl...|
+--------------------+



In [120]:

person=person.fillna(value='Unspecified',subset=["PED_LOCATION"])

In [121]:
person.select("PED_ACTION").distinct().show()


+--------------------+
|          PED_ACTION|
+--------------------+
|Pushing/Working o...|
|      Not in Roadway|
|Riding/Walking Al...|
|Crossing, No Sign...|
|      Does Not Apply|
|                null|
|Other Actions in ...|
|             Unknown|
|  Working in Roadway|
|Getting On/Off Ve...|
|Emerging from in ...|
|  Playing in Roadway|
|Crossing Against ...|
|Crossing, No Sign...|
|Going to/From Sto...|
|Crossing With Signal|
|Riding/Walking Al...|
+--------------------+



In [123]:
person=person.fillna(value='Unknown',subset=["PED_ACTION"])

In [124]:
person.select("COMPLAINT").distinct().show()

+--------------------+
|           COMPLAINT|
+--------------------+
|   Complaint of Pain|
|          Minor Burn|
|  Contusion - Bruise|
|      Does Not Apply|
|         Severe Burn|
|                null|
|      Minor Bleeding|
|      Crush Injuries|
|           Paralysis|
|             Unknown|
|            Abrasion|
|Fracture - Disloc...|
|        None Visible|
|            Internal|
|            Whiplash|
|Fracture - Distor...|
|          Concussion|
|     Severe Bleeding|
|  Severe Lacerations|
|Complaint of Pain...|
+--------------------+
only showing top 20 rows



In [139]:
person=person.fillna(value='Unspecified',subset=["COMPLAINT"])

In [140]:
person.select("PED_ROLE").distinct().show()

+---------------+
|       PED_ROLE|
+---------------+
|        Witness|
|         Driver|
|Notified Person|
|  Policy Holder|
|        Unknown|
|          Other|
|      Passenger|
|     Registrant|
|     Pedestrian|
|          Owner|
| In-Line Skater|
+---------------+



In [141]:
person=person.fillna(value='Unknown',subset=["PED_ROLE"])

In [142]:
person.select("CONTRIBUTING_FACTOR_1").distinct().show(100)


+---------------------+
|CONTRIBUTING_FACTOR_1|
+---------------------+
| Following Too Clo...|
| Traffic Control D...|
| Driverless/Runawa...|
| Windshield Inadeq...|
| Using On Board Na...|
|         Unsafe Speed|
| Shoulders Defecti...|
|       Tinted Windows|
|    Oversized Vehicle|
| Passing or Lane U...|
| Lane Marking Impr...|
| Other Lighting De...|
| Aggressive Drivin...|
|      Other Vehicular|
|  Driver Inexperience|
|              Texting|
|               Illnes|
|      Drugs (illegal)|
|          Unspecified|
|   Pavement Defective|
|              Unknown|
| Prescription Medi...|
| View Obstructed/L...|
|   Lost Consciousness|
| Reaction to Uninv...|
|          Fell Asleep|
| Tire Failure/Inad...|
| Outside Car Distr...|
|      Fatigued/Drowsy|
| Driver Inattentio...|
| Cell Phone (hand-...|
| Listening/Using H...|
| Failure to Keep R...|
|   Obstruction/Debris|
| Unsafe Lane Changing|
| Pedestrian/Bicycl...|
| Passenger Distrac...|
|  Alcohol Involvement|
| Traffic Contro

In [143]:
person=person.fillna(value='Unknown',subset=["CONTRIBUTING_FACTOR_1"])

In [144]:
person.select("CONTRIBUTING_FACTOR_2").distinct().show(100)

+---------------------+
|CONTRIBUTING_FACTOR_2|
+---------------------+
| Following Too Clo...|
| Traffic Control D...|
| Driverless/Runawa...|
| Accelerator Defec...|
|         Unsafe Speed|
|    Oversized Vehicle|
| Passing or Lane U...|
| Lane Marking Impr...|
| Aggressive Drivin...|
|      Other Vehicular|
|  Driver Inexperience|
|              Texting|
|               Illnes|
|      Drugs (illegal)|
|          Unspecified|
|   Pavement Defective|
|              Unknown|
| Prescription Medi...|
| View Obstructed/L...|
|   Lost Consciousness|
| Reaction to Uninv...|
|          Fell Asleep|
| Tire Failure/Inad...|
| Outside Car Distr...|
|      Fatigued/Drowsy|
| Driver Inattentio...|
| Cell Phone (hand-...|
| Listening/Using H...|
| Failure to Keep R...|
|   Obstruction/Debris|
| Unsafe Lane Changing|
| Pedestrian/Bicycl...|
| Passenger Distrac...|
|  Alcohol Involvement|
| Traffic Control D...|
|     Backing Unsafely|
|     Brakes Defective|
|   Eating or Drinking|
| Failure to Yie

In [145]:
person.select("PED_ROLE").distinct().show()

+---------------+
|       PED_ROLE|
+---------------+
|        Witness|
|         Driver|
|Notified Person|
|  Policy Holder|
|        Unknown|
|          Other|
|      Passenger|
|     Registrant|
|     Pedestrian|
|          Owner|
| In-Line Skater|
+---------------+



In [146]:
person=person.fillna(value='Unknown',subset=["PED_ROLE"])

In [147]:
person.select("CONTRIBUTING_FACTOR_1").distinct().show(100)

+---------------------+
|CONTRIBUTING_FACTOR_1|
+---------------------+
| Following Too Clo...|
| Traffic Control D...|
| Driverless/Runawa...|
| Windshield Inadeq...|
| Using On Board Na...|
|         Unsafe Speed|
| Shoulders Defecti...|
|       Tinted Windows|
|    Oversized Vehicle|
| Passing or Lane U...|
| Lane Marking Impr...|
| Other Lighting De...|
| Aggressive Drivin...|
|      Other Vehicular|
|  Driver Inexperience|
|              Texting|
|               Illnes|
|      Drugs (illegal)|
|          Unspecified|
|   Pavement Defective|
|              Unknown|
| Prescription Medi...|
| View Obstructed/L...|
|   Lost Consciousness|
| Reaction to Uninv...|
|          Fell Asleep|
| Tire Failure/Inad...|
| Outside Car Distr...|
|      Fatigued/Drowsy|
| Driver Inattentio...|
| Cell Phone (hand-...|
| Listening/Using H...|
| Failure to Keep R...|
|   Obstruction/Debris|
| Unsafe Lane Changing|
| Pedestrian/Bicycl...|
| Passenger Distrac...|
|  Alcohol Involvement|
| Traffic Contro

In [148]:
person=person.fillna(value='Unknown',subset=["CONTRIBUTING_FACTOR_1"])

In [149]:
person.select("CONTRIBUTING_FACTOR_2").distinct().show(100)

+---------------------+
|CONTRIBUTING_FACTOR_2|
+---------------------+
| Following Too Clo...|
| Traffic Control D...|
| Driverless/Runawa...|
| Accelerator Defec...|
|         Unsafe Speed|
|    Oversized Vehicle|
| Passing or Lane U...|
| Lane Marking Impr...|
| Aggressive Drivin...|
|      Other Vehicular|
|  Driver Inexperience|
|              Texting|
|               Illnes|
|      Drugs (illegal)|
|          Unspecified|
|   Pavement Defective|
|              Unknown|
| Prescription Medi...|
| View Obstructed/L...|
|   Lost Consciousness|
| Reaction to Uninv...|
|          Fell Asleep|
| Tire Failure/Inad...|
| Outside Car Distr...|
|      Fatigued/Drowsy|
| Driver Inattentio...|
| Cell Phone (hand-...|
| Listening/Using H...|
| Failure to Keep R...|
|   Obstruction/Debris|
| Unsafe Lane Changing|
| Pedestrian/Bicycl...|
| Passenger Distrac...|
|  Alcohol Involvement|
| Traffic Control D...|
|     Backing Unsafely|
|     Brakes Defective|
|   Eating or Drinking|
| Failure to Yie

In [150]:
person=person.fillna(value='Unknown',subset=["CONTRIBUTING_FACTOR_2"])

In [151]:
#Change CRASH_DATE from String to date
from pyspark.sql.types import DataType
from pyspark.sql.functions import to_date

person = person.withColumn('CRASH_DATE', to_date(person["CRASH_DATE"], 'MM/dd/yyyy'))
person.printSchema()


root
 |-- UNIQUE_ID: integer (nullable = true)
 |-- COLLISION_ID: integer (nullable = true)
 |-- CRASH_DATE: date (nullable = true)
 |-- CRASH_TIME: string (nullable = true)
 |-- PERSON_ID: string (nullable = false)
 |-- PERSON_TYPE: string (nullable = true)
 |-- PERSON_INJURY: string (nullable = true)
 |-- VEHICLE_ID: integer (nullable = true)
 |-- PERSON_AGE: integer (nullable = true)
 |-- EJECTION: string (nullable = false)
 |-- EMOTIONAL_STATUS: string (nullable = false)
 |-- BODILY_INJURY: string (nullable = false)
 |-- POSITION_IN_VEHICLE: string (nullable = false)
 |-- SAFETY_EQUIPMENT: string (nullable = false)
 |-- PED_LOCATION: string (nullable = false)
 |-- PED_ACTION: string (nullable = false)
 |-- COMPLAINT: string (nullable = false)
 |-- PED_ROLE: string (nullable = false)
 |-- CONTRIBUTING_FACTOR_1: string (nullable = false)
 |-- CONTRIBUTING_FACTOR_2: string (nullable = false)
 |-- PERSON_SEX: string (nullable = false)



In [152]:
from pyspark.sql.functions import to_timestamp
person = person.withColumn('CRASH_TIME', to_timestamp(person["CRASH_TIME"], 'H:mm'))
person.printSchema()

root
 |-- UNIQUE_ID: integer (nullable = true)
 |-- COLLISION_ID: integer (nullable = true)
 |-- CRASH_DATE: date (nullable = true)
 |-- CRASH_TIME: timestamp (nullable = true)
 |-- PERSON_ID: string (nullable = false)
 |-- PERSON_TYPE: string (nullable = true)
 |-- PERSON_INJURY: string (nullable = true)
 |-- VEHICLE_ID: integer (nullable = true)
 |-- PERSON_AGE: integer (nullable = true)
 |-- EJECTION: string (nullable = false)
 |-- EMOTIONAL_STATUS: string (nullable = false)
 |-- BODILY_INJURY: string (nullable = false)
 |-- POSITION_IN_VEHICLE: string (nullable = false)
 |-- SAFETY_EQUIPMENT: string (nullable = false)
 |-- PED_LOCATION: string (nullable = false)
 |-- PED_ACTION: string (nullable = false)
 |-- COMPLAINT: string (nullable = false)
 |-- PED_ROLE: string (nullable = false)
 |-- CONTRIBUTING_FACTOR_1: string (nullable = false)
 |-- CONTRIBUTING_FACTOR_2: string (nullable = false)
 |-- PERSON_SEX: string (nullable = false)



In [153]:
from pyspark.sql.functions import date_format
from pyspark.sql import functions as f
person = person.withColumn('CRASH_TIME', f.date_format("CRASH_TIME", 'HH:mm'))

In [154]:
person.select("CRASH_TIME").show(10)

+----------+
|CRASH_TIME|
+----------+
|     09:43|
|     15:15|
|     17:55|
|     13:05|
|     11:16|
|     19:15|
|     16:45|
|     01:15|
|     13:04|
|     08:20|
+----------+
only showing top 10 rows



In [155]:
person.select("CRASH_DATE").show(10)

+----------+
|CRASH_DATE|
+----------+
|2019-10-26|
|2019-10-25|
|2019-10-26|
|2016-11-21|
|2019-10-25|
|2019-10-24|
|2019-10-26|
|2019-10-26|
|2019-10-26|
|2019-10-24|
+----------+
only showing top 10 rows



In [156]:
person.repartition(1).write.format('com.databricks.spark.csv').save("./person_clean.csv",header = 'true')

In [157]:
col.repartition(1).write.format('com.databricks.spark.csv').save("./collision_clean.csv",header = 'true')